In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004183292388916016
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 22.0445689174115
200000 32.26499109671786
300000 31.808430172079476
400000 31.24437866183498
500000 32.61745977336206
600000 31.19477691428588
700000 30.70314834746807
800000 31.191925794838458
900000 26.027813028156586
1000000 17.66079786941932
1100000 30.529579575428535
1200000 30.61630376310348
1300000 30.57917631331736
1400000 32.29540237955594
1500000 30.943080778639846
1600000 30.626605583202547
1700000 31.064165880941033
1800000 28.87641454781971
1900000 13.939380204335677
2000000 29.666396011132765
2100000 29.298391797009593
2200000 28.892323118886537
2300000 29.903128455140653
2400000 29.07253475212184
2500000 28.61525022152903
2600000 28.748780371085793
2700000 28.230472106773323
2800000 12.75614874737903
2900000 28.943142661278248
3000000 28.17

29400000 22.275470345438173
29500000 30.547963412857513
29600000 28.495349530917228
29700000 28.239853165567812
29800000 26.8273301240313
29900000 26.244692778164318
30000000 11.99975616770564
30100000 26.792713274506898
30200000 26.099153701049815
30300000 27.87272495876191
30400000 28.492239740179368
30500000 29.927690092345536
30600000 29.169019236200377
30700000 15.216543973918348
30800000 29.206527777180792
30900000 29.657924681963827
31000000 29.001428422215927
31100000 28.201892574937126
31200000 14.710594681718502
31300000 29.266603017615836
31400000 27.743888419469567
31500000 28.47197723862516
31600000 26.49409420398096
31700000 24.876724265323887
31800000 14.93011752075957
31900000 22.383529540132866
32000000 24.87363427770839
32100000 27.382786580771114
32200000 27.17178254866299
32300000 28.5694806457448
32400000 29.418502403694173
32500000 14.852895955609757
32600000 28.243799344889485
32700000 28.99948330054699
32800000 28.915812768348008
32900000 29.70691571134443
33000

59100000 9.262628746601257
59200000 16.773479775979045
59300000 17.095243190463872
59400000 13.330124632940866
59500000 7.742483444119576
59600000 16.236631688512905
59700000 16.44157489183637
59800000 15.77366302953713
59900000 13.023894850564172
60000000 15.354687860888616
60100000 13.140111658782699
60200000 15.69148144512719
60300000 11.957388380165597
60400000 12.351362793671976
60500000 10.82496321161446
60600000 18.503322538601715
60700000 16.827309134268262
60800000 12.113134914901963
60900000 5.028956485427585
61000000 13.938783678487686
61100000 14.697134281984807
61200000 18.054762604976737
61300000 8.613454657987049
61400000 19.202014464624483
61500000 18.324814342144794
61600000 17.54620972117164
61700000 13.580707222106287
61800000 22.460855680706846
61900000 16.4249353427817
62000000 16.62512856476161
62100000 16.09155141003892
62200000 16.495743351697186
62300000 6.969911469435314
62400000 12.096103820983807
62500000 13.679645950866766
62600000 16.88650658175281
6270000

88800000 25.63039154018401
88900000 26.55251503232169
89000000 26.438798687398645
89100000 26.030819354789998
89200000 25.011460404620127
89300000 23.63257535581843
89400000 22.763598210498603
89500000 24.431264467113717
89600000 22.58555392784643
89700000 19.3555081553972
89800000 15.56697547398893
89900000 15.692078196835487
90000000 18.915930271330488
90100000 13.223967304753796
90200000 23.198572737016214
90300000 23.889984326731483
90400000 25.13053975836907
90500000 23.84854152279843
90600000 23.301508209721543
90700000 24.92377375949118
90800000 25.6396398002589
90900000 26.734084487187335
91000000 25.529512307192334
91100000 24.196261039646664
91200000 23.309215108375014
91300000 23.945879772722193
91400000 23.150869030787927
91500000 23.7393531033247
91600000 11.667185461274693
91700000 18.142585118749558
91800000 15.311198780255976
91900000 13.089972084756953
92000000 24.195782908992477
92100000 24.697703005099235
92200000 23.7574989742088
92300000 23.702430077157935
92400000

117900000 30.721417360874312
118000000 31.846587226722832
118100000 28.849384289707324
118200000 25.08142844134056
118300000 24.200088063597065
118400000 22.986856429956475
118500000 24.486703435256608
118600000 24.065488142538072
118700000 23.919787089363933
118800000 23.030181481685965
118900000 15.543386006429387
119000000 3.8017356415621935
119100000 16.448585055931517
119200000 22.97729909333358
119300000 25.53847564039114
119400000 24.496323858364075
119500000 24.544850193342086
119600000 23.748536350399554
119700000 26.575730704158858
119800000 25.83863268481319
119900000 26.29647824184304
120000000 22.65237318205253
120100000 22.151372174798173
120200000 21.452607575692692
120300000 20.644338331950024
120400000 20.838161022532244
120500000 20.864601261574833
120600000 19.871406222963625
120700000 14.494796030520396
120800000 3.807998791675965
120900000 11.070822133627756
121000000 20.26801943483868
121100000 21.08783887284782
121200000 20.32139179422535
121300000 21.96588888846

...

320200000 4.930615925931518
320300000 4.826995403027684
320400000 5.111539893051663
320500000 2.2403092007131677
320600000 2.60744673004965
320700000 5.164273715809011
320800000 5.210793265783284
320900000 5.4401800457223155
321000000 5.376892477874646
321100000 5.4677815877208475
321200000 6.0372820429298875
321300000 5.680921265272177
321400000 5.888738212078912
321500000 5.4693092039075095
321600000 2.803110534811043
321700000 4.263255746897029
321800000 4.9633636855748495
321900000 4.9724405526902355
322000000 5.241272983003129
322100000 5.038243109599888
322200000 4.631996329191086
322300000 3.0123204659255083
322400000 1.6445604995782475
322500000 5.553616733234141
322600000 5.448309326284634
322700000 5.313783068065474
322800000 5.3847104681895805
322900000 5.649855098478589
323000000 5.65656126901558
323100000 5.152616689543399
323200000 6.075177455356244
323300000 5.248672816894343
323400000 2.5707268947954542
323500000 4.610042851728423
323600000 4.959645708061287
323700000 4

348400000 3.4395877264845964
348500000 2.9480756002058586
348600000 3.8185919800530277
348700000 3.943966020995375
348800000 4.006583965182165
348900000 3.0733434515953295
349000000 3.3660020757024323
349100000 3.035272894187987
349200000 2.700813183381644
349300000 3.6067432785893807
349400000 2.577073343412906
349500000 4.080172147904671
349600000 3.935531843910184
349700000 3.8696345182850655
349800000 3.140482097772317
349900000 3.8807802674236265
350000000 3.1603974837180377
350100000 2.642421799223811
350200000 3.5630919932059344
350300000 2.5749407467439474
350400000 3.8148216343384482
350500000 3.8872541284889186
350600000 3.726067305080474
350700000 3.131182552574254
350800000 3.6904514005065856
350900000 3.3845935200588397
351000000 2.2845663604560795
351100000 3.210701635784053
351200000 2.4928138833854496
351300000 3.735636553778755
351400000 3.9357276551980904
351500000 3.7946998493931092
351600000 3.414523977156794
351700000 3.162222251057419
351800000 3.440547734835833
3

377100000 1.581712495670802
377200000 2.090208588045011
377300000 1.3457348421027362
377400000 2.1781038986056642
377500000 1.685482918270939
377600000 2.239167538897872
377700000 2.2837221396753296
377800000 2.0985344103094836
377900000 2.062634730252653
378000000 1.5947652251449327
378100000 2.0576725400556457
378200000 1.204854197248588
378300000 1.679865074633426
378400000 1.4289314752459361
378500000 2.0926856462157866
378600000 2.085231282745944
378700000 2.277058918066038
378800000 2.0971722959620056
378900000 1.529578720997158
379000000 2.1277071859062975
379100000 1.1614897891634404
379200000 2.0537117513525485
379300000 1.6943914804202618
379400000 2.107942198290924
379500000 2.19948249467859
379600000 1.6740857991604143
379700000 1.9045651284564435
379800000 1.2389049021760559
379900000 1.6771050473924478
380000000 1.2972645930620332
380100000 2.102747409648584
380200000 1.7609890607464942
380300000 1.9733948217173913
380400000 1.7351737120898365
380500000 2.1399297533764807

404300000 0.5407596674664914
404400000 0.3942759912900262
404500000 0.5438176776571902
404600000 0.6327137099571228
404700000 0.8125285551746421
404800000 0.4439700982218915
404900000 0.3821510478846431
405000000 0.6360889580800434
405100000 0.625774950078792
405200000 1.0438802295860152
405300000 0.3619419307952921
405400000 0.5140133826015493
405500000 0.6035239732252956
405600000 0.5889019924193952
405700000 0.47723338141333393
405800000 0.333522619844079
405900000 0.6213588390297625
406000000 0.5881531199256116
406100000 0.576063046762079
406200000 0.3325160513533652
406300000 0.9482706918312178
406400000 0.5913236128934821
406500000 0.75824150665465
406600000 0.5263410776884092
406700000 0.3575591255704164
406800000 0.5519491986003485
406900000 0.39097342470820745
407000000 0.12221042624141773
407100000 0.12097383301094175
407200000 0.12164896771902507
407300000 0.11976598753663899
407400000 0.11833448228212197
407500000 0.11957862864429089
407600000 0.1198206626187265
407700000 0

430600000 0.1273661190445357
430700000 0.10188453907164932
430800000 0.09759392461640304
430900000 0.13741568355824219
431000000 0.19131181529751418
431100000 0.06988094571363263
431200000 0.13640568763387864
431300000 0.06884568206994733
431400000 0.09175675510972076
431500000 0.07390780233758025
431600000 0.08580845533004404
431700000 0.11110381434615121
431800000 0.068575031776766
431900000 0.08712695724579693
432000000 0.061259986866755614
432100000 0.08885914813993374
432200000 0.06638440484998624
432300000 0.09692601279135875
432400000 0.07651006749610106
432500000 0.06760005625767508
432600000 0.12702293403370513
432700000 0.06322466570300526
432800000 0.0664059585686326
432900000 0.059696407390574614
433000000 0.06491034491941333
433100000 0.09289938039719063
433200000 0.06986033149267237
433300000 0.05291105610311032
433400000 0.043802257689644894
433500000 0.07647435816026728
433600000 0.06956808151532876
433700000 0.062072230358719824
433800000 0.042834959544082486
433900000